**photo->text->OCR->Extracted text-->Chemical Extraction-> predict through csv**

In [ ]:
import joblib

model = joblib.load("majority_ensemble.pkl")
label_encoder = joblib.load("label_encoder.pkl")

In [1]:
import pytesseract
import cv2
import pubchempy as pcp
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\dhruv\AppData\Local\Programs\Tesseract-OCR\tesseract.exe' #tesseract path

In [3]:
#image then convert it into 
image = cv2.imread('Screenshot 2025-06-08 161620.png')  
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# remove noise and improve clarity
gray = cv2.GaussianBlur(gray, (3, 3), 0)

In [4]:
#Ocr: it will extract text from img
text = pytesseract.image_to_string(gray)

print("Extracted Ingredients:")
print(len(text))

Extracted Ingredients:
964


In [5]:
import re

def preprocess_text(text):
    # Remove special characters, digits, extra whitespace
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip().lower()
    return text
preprocess_text(text)
print(len(text))

964


In [6]:
text

'CALIFORNIA PROPOSITION 65\nWarning: This product contoins nicotine, o chemical known to the\nstote of Colifornio to couse birth defects or other reproductive horm\n\nTngr “ym Silas Hota. Nicotine, FCC Grade Vegetable Glycerin,\n\n[etre Fos, lv Ac. Westin conten 6 8 per\ncontri\n\nMade in China.\nFlavors mode in USA\n©2012 LOEC, in.\n\nblu™ ond blu eCigs® are trademarks of Lorillord Technologies, Inc.\n\n1 serving per pack\nServing size 1 pack (32.35)\ncesar appara\n\nCalories 150\n\nTrans Fat 09\nCholesterol m9\nSodium 1059 ot\n\nINGREDIENTS\nMILK CHOCOLATE (SUGAR\nCHOCOLATE MILK. COCOA\nBL T T. SOY\nLE\n\n(WHEAT FLOUR. NIACIN\n\nIRON, AMINE M\n\nRIBOFLAVIN, FOLIC ACID)\n\nMODIFIED SAGO STARCH. SAI\n\nSOYBEAN OIL, CORN SYRUP.\n\nBARLEY MALT EXTRACT\n\nLEAVENING (BAKING SODA. YEAS\n\nAND/OR AMMONIUM\n\nBICARBONATE)]. LESS THAN 2\n\nCORNSTARCH, CORN SYRUP.\n\nDEXTRIN. SALT. COLORING\n\n(INCLUDES BLUE 1 LAKE, RED 40.\nLLOW 6, YELLOW S$. BLUE 1, RED\n\n40 LAKE, YELLOW 6 LAK!\n\nSLAKE\n\nC

In [7]:
import joblib
model = joblib.load("Tuned_model.joblib")
vectorizer = joblib.load("tfidf_vectorizer.joblib")

# model = joblib.load("Tuned_model.joblib_recent")
# vectorizer = joblib.load("tfidf_vectorizer.joblib_recent")

C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dhruv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:440: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransform

In [8]:
def predict_product_category(text):
    clean_text = preprocess_text(text)
    X_vec = vectorizer.transform([clean_text])
    predicted_label = model.predict(X_vec)[0]
    return predicted_label


In [9]:
ocr_output_text = "Aqua, Glycerin, Paraben, Fragrance"
ocr_output2 = "Calcium Carbonate, Microcrystalline Cellulose, Talc, Magnesium Stearate"

predicted_category = predict_product_category(text)
print("Predicted Category:", predicted_category)
# chemical_list = ["Paraben", "Fragrance", "Glycerin"]
# for chem in chemical_list:
#     score = get_contextual_harm_score(chem, predicted_category)
#     print(f"{chem} in {predicted_category} → Harm Score: {score}%")


Predicted Category: 2


In [10]:
#for others 

def predict_category_with_fallback(text, model, vectorizer, threshold=0.45):
    clean_text = preprocess_text(text)
    X_vec = vectorizer.transform([clean_text])
    
    proba = model.predict_proba(X_vec)[0]
    max_confidence = np.max(proba)
    predicted_label = model.classes_[np.argmax(proba)]

    # Fallback to "others" if confidence is too low
    if max_confidence < threshold:
        return "others",proba
    return predicted_label,proba

In [11]:
ocr_output = "Aqua, Glycerin, Paraben, Fragrance"
ocr_output2 = "Calcium Carbonate, Microcrystalline Cellulose, Talc, Magnesium Stearate"

category, proba = predict_category_with_fallback(ocr_output2, model, vectorizer)
print("Predicted Category:", category)

Predicted Category: others


In [12]:
print("Confidence Scores:", dict(zip(model.classes_, proba)))


Confidence Scores: {0: 0.06826293469599233, 1: 0.15197334223267034, 2: 0.05653453602289769, 3: 0.16256544994690067, 4: 0.1473550651538402, 5: 0.2140389148476442, 6: 0.10310496088536648, 7: 0.09616479621468824}


In [13]:
##chemical


In [14]:
#we will use NLp Spacy for Chemical extraction
import spacy
nlp = spacy.load("en_ner_bc5cdr_md")

# text = "Sodium Lauryl Sulfate, Propylparaben, Aloe Vera"
doc = nlp(text)

chemicals = [ent.text for ent in doc.ents if ent.label_ == "CHEMICAL"]
print(f"Extracted chemicals: {chemicals}")


PydanticImportError: `pydantic:ConstrainedStr` has been removed in V2.

For further information visit https://errors.pydantic.dev/2.11/u/import-error

In [ ]:
#from chemicals now we filtered chemicals : chemicals no noise
import pubchempy as pcp

raw_chemicals = [ent.text for ent in doc.ents if ent.label_ == "CHEMICAL"]

# Validate with PubChem
def is_valid_chemical(name):
    try:
        result = pcp.get_compounds(name, 'name')
        return len(result) > 0
    except:
        return False

# Filter only valid chemical names
filtered_chemicals = [chem for chem in raw_chemicals if is_valid_chemical(chem)]

print("Extracted Chemicals:", filtered_chemicals)



In [ ]:
# #we have to use model here for prediction after getting chemicals 

# predicted_category = predict_product_category(filtered_chemicals)
# print("Predicted Category:", predicted_category)

**harmfull prediction**

In [ ]:
chem=pd.read_csv("Chemicals_label.csv")


In [57]:
#function for harmfull chemicals : among chemicals which one is chemical
def get_harm_info(chemical_name):
    row=chem[chem["chemical_name"].str.lower()==chemical_name.lower()]

    if not row.empty:
        harm_score=int(row["harm_score"].values[0])
        #no alternative values
        return harm_score

    else:
        #chemicals name not found return Safe 
        return 0,None

In [58]:
#integration with chemical 
overall = []
count = 0

for chemical in filtered_chemicals:
    harm_score = get_harm_info(chemical)

    #  harm_score is a tuple 
    if isinstance(harm_score, tuple) and len(harm_score) > 0:
        harm_score_value = harm_score[0]  # Extract the first element of the tuple
    else:
        harm_score_value = harm_score  # In case it's already an integer or float

    if harm_score_value > 0:
        overall.append(harm_score_value)
        count += 1

    print(f"Chemical: {chemical}")
    print(f"Harmfulness Score: {harm_score_value}%")

if count > 0:
    average = sum(overall) / count
    print(f"Average Harmfulness Score: {average}%")
else:
    print("No harmful chemicals found to calculate an average.")


Chemical: Formaldehyde
Harmfulness Score: 95%
Chemical: Ethanolamine
Harmfulness Score: 0%
Average Harmfulness Score: 95.0%


In [59]:
#ai detection#

In [60]:
import csv

input_txt_file = 'extended_product_dataset.txt'
output_csv_file = 'product.csv.csv'

# Reading the text file
with open(input_txt_file, 'r') as txtfile:
    lines = txtfile.readlines()

# Writing to the CSV file
with open(output_csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    for line in lines:
        # Assuming the values are separated by commas
        row = line.strip().split(',')
        writer.writerow(row)

print("Text file has been converted to CSV successfully.")



FileNotFoundError: [Errno 2] No such file or directory: 'extended_product_dataset.txt'

In [15]:
#Photo → OCR → Text → Category Prediction → Chemicals Extraction → Contextual Harm Scoring


In [16]:
ocr_output = "nicotine, Nicotine, FCC, Glycerin, Cholesterol, Sodium"

In [1]:
label_encoder = joblib.load("label_encoder.pkl")

def preprocess_text(text):
    return text.lower().replace(",", " ")

def predict_category_with_fallback(text, model, vectorizer, label_encoder, threshold=0.45):
    clean_text = preprocess_text(text)
    X_vec = vectorizer.transform([clean_text])
    
    proba = model.predict_proba(X_vec)[0]
    max_confidence = np.max(proba)
    predicted_index = np.argmax(proba)
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]

    if max_confidence < threshold:
        return "others", max_confidence
    return predicted_label, max_confidence


category, confidence = predict_category_with_fallback(ocr_output, model, vectorizer, label_encoder)
print(f"Predicted Category: {category} | Confidence: {confidence:.2f}")


NameError: name 'ocr_output' is not defined

In [18]:
examples = [
    "Aqua, Glycerin, Paraben, Fragrance, Dimethicone",
    "Sodium Benzoate, Citric Acid, Natural Flavours, Xanthan Gum, Sugar",
    "Water, Sodium Laureth Sulfate, Cocamidopropyl Betaine, PEG-40, Perfume",
    "Paracetamol, Caffeine, Starch, Povidone, Magnesium Stearate",
    "Sodium Hypochlorite, Limonene, Alcohol Ethoxylate, Sodium Carbonate, Water",
    "Polyvinyl Alcohol, Acetone, Isopropyl Alcohol, Ethylene Glycol, Pigment Red 22",
    "Citronellol, Linalool, D-Limonene, Butylphenyl Methylpropional, Coumarin",
    "Polystyrene, Formaldehyde, Ethanolamine, Dipropylene Glycol, BHT"
]

for text in examples:
    category, confidence = predict_category_with_fallback(text, model, vectorizer, label_encoder)
    print(f"Text: {text}\n → Predicted Category: {category} | Confidence: {confidence:.2f}\n")


Text: Aqua, Glycerin, Paraben, Fragrance, Dimethicone
 → Predicted Category: others | Confidence: 0.18

Text: Sodium Benzoate, Citric Acid, Natural Flavours, Xanthan Gum, Sugar
 → Predicted Category: food | Confidence: 0.88

Text: Water, Sodium Laureth Sulfate, Cocamidopropyl Betaine, PEG-40, Perfume
 → Predicted Category: personal_care | Confidence: 0.82

Text: Paracetamol, Caffeine, Starch, Povidone, Magnesium Stearate
 → Predicted Category: others | Confidence: 0.20

Text: Sodium Hypochlorite, Limonene, Alcohol Ethoxylate, Sodium Carbonate, Water
 → Predicted Category: cleaning | Confidence: 0.77

Text: Polyvinyl Alcohol, Acetone, Isopropyl Alcohol, Ethylene Glycol, Pigment Red 22
 → Predicted Category: stationery | Confidence: 0.94

Text: Citronellol, Linalool, D-Limonene, Butylphenyl Methylpropional, Coumarin
 → Predicted Category: others | Confidence: 0.37

Text: Polystyrene, Formaldehyde, Ethanolamine, Dipropylene Glycol, BHT
 → Predicted Category: others | Confidence: 0.35



In [1]:
#removing some columns stationery personal care and remove other and if confidence is below threshold it will go under others

In [2]:
#predicting harmscore based on category

In [23]:
def calculate_harm_score(chemicals, category, harm_scores_csv):
    harm_df = pd.read_csv(harm_scores_csv)
    
    # Column for the specific category
    category_col = f"harm_score_{category}"
    if category_col not in harm_df.columns:
        raise ValueError(f"Category '{category}' not found in harm score table.")

    scores = []
    for chem in chemicals:
        row = harm_df[harm_df['chemical_name'].str.lower() == chem.lower()]
        if not row.empty:
            score = row[category_col].values[0]
            scores.append(score)

    if not scores:
        return None, None  # No chemicals matched

    avg_score = sum(scores) / len(scores)

    # Risk Level
    if avg_score <= 30:
        risk = "Low Risk"
    elif avg_score <= 70:
        risk = "Moderate Risk"
    else:
        risk = "High Risk"

    return avg_score, risk


chemicals_list = ["Sodium Benzoate", "Citric Acid"," Natural Flavours", "Xanthan Gum", "Sugar"]  # From OCR
predicted_category = "cosmetic"

avg_score, risk_level = calculate_harm_score(
    chemicals_list, 
    predicted_category, 
    "chemical_harmness_category.csv"
)

print(f"Average Harm Score: {avg_score:.2f}%")
print(f"Risk Level: {risk_level}")
print(f"Category :{category}")

Average Harm Score: 20.00%
Risk Level: Low Risk
Category :others
